<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/aya_expanse_8b/multigec/multigec_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

Cloning into 'multigec-models'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 115 (delta 34), reused 66 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 141.95 KiB | 1.80 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [4]:
%cd multigec-models

/content/multigec-models


In [5]:
!git pull

Already up to date.


In [6]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade transformers trl -q
!pip install pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [8]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

from peft import PeftModel, PeftModelForCausalLM

tqdm.pandas()

In [9]:
parameters = OmegaConf.load("./params/aya_expanse_8b.yaml")

In [10]:
track     = "minimal"
fine_tuned_model_name = f"aya-expanse-8b-multigec"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")

in_path  = f"/content/drive/MyDrive/multigec/datasets/multigec_{track}.csv"
out_path = f"/content/drive/MyDrive/multigec/preds/multigec_test_{track}_chunked.csv"

out_model_dir = f"/content/drive/MyDrive/multigec/models/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [11]:
login(hf_key)

In [12]:
!env TORCH_USE_CUDA_DSA=1 -q

env: ‘-q’: No such file or directory


In [15]:
base_model = "CohereForAI/aya-expanse-8b"
saved_checkpoint = out_model_dir + "/checkpoint-700"

quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(saved_checkpoint)
config = AutoConfig.from_pretrained(base_model)
base_model_instance = AutoModelForCausalLM.from_pretrained(
    base_model,
    config=config,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map=device,
    attn_implementation="flash_attention_2",
)
base_model_instance.resize_token_embeddings(len(tokenizer))
model = PeftModelForCausalLM.from_pretrained(
    base_model_instance,
    saved_checkpoint,
    torch_dtype=torch.bfloat16,
    device_map=device,
    ignore_mismatched_sizes=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:470: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated
- base_model.model.model.embed_tokens.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated.
  warnings.warn(msg)


In [16]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

In [17]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>"

    return text

In [96]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from wtpsplit import SaT

# ------------------------------------------------------------------------------
# ASSUMPTIONS:
# 1) You already have:
#       model         -> Your loaded model
#       tokenizer     -> Your tokenizer
#       test_df       -> A DataFrame with at least columns ["text", "target"]
#       out_path      -> Path to save final CSV
# 2) You want to process only rows where "target" is NaN, splitting the "text"
#    if it exceeds a certain max_tokens threshold, then recombining outputs.
# ------------------------------------------------------------------------------

# Instantiate the sentence splitter (wtpsplit)
sat = SaT()

# Adjustable parameters
max_new_tokens = 600
batch_size = 20
save_each = 20
max_tokens = 600  # maximum tokens allowed per chunk prompt

# ------------------------------------------------------------------------------
# 1) Function to chunk large texts
#    Splits first by newlines, then by sentences, then by spaces if still too big.
# ------------------------------------------------------------------------------
def chunk_large_texts(df, tokenizer, text_col="text", max_tokens=2048):
    """
    Given a DataFrame `df` with a column `text_col`, split each text if it exceeds
    `max_tokens`. Return a new DataFrame with multiple rows (chunks) per original row.
    We preserve the original row index in 'index' (from df) and add a 'chunk_id'.
    """
    chunked_rows = []

    for i, row in df.iterrows():
        original_df_index = row["index"]  # the original test_df index
        full_text = row[text_col]

        # Quickly check if it needs chunking
        initial_ids = tokenizer(full_text, add_special_tokens=False).input_ids
        if len(initial_ids) < max_tokens:
            # No chunking needed
            new_row = row.copy()
            new_row["chunk_id"] = 0
            chunked_rows.append(new_row)
            continue

        # Otherwise, we chunk
        splitted_texts = full_text.split("\n")

        all_chunks = []
        for s in splitted_texts:
            source_inputs = tokenizer(s, add_special_tokens=False).input_ids
            if len(source_inputs) < max_tokens:
                all_chunks.append(s)
            else:
                # Split by sentences using SaT
                sent_splits = sat.split(s)
                curr_input = ""

                for sent in sent_splits:
                    candidate = (curr_input + " " + sent).strip()
                    curr_tokens = tokenizer(candidate, add_special_tokens=False).input_ids

                    if len(curr_tokens) < max_tokens:
                        curr_input = candidate
                    else:
                        # Store what we had so far
                        if curr_input:
                            all_chunks.append(curr_input)

                        # Now chunk 'sent' by spaces if it's still too big
                        words = sent.split()
                        sub_chunk = []
                        for w in words:
                            test_sub = " ".join(sub_chunk + [w])
                            test_sub_ids = tokenizer(test_sub, add_special_tokens=False).input_ids
                            if len(test_sub_ids) < max_tokens:
                                sub_chunk.append(w)
                            else:
                                # store the chunk we had
                                if sub_chunk:
                                    all_chunks.append(" ".join(sub_chunk))
                                # start new chunk
                                sub_chunk = [w]

                        # flush remainder
                        if sub_chunk:
                            all_chunks.append(" ".join(sub_chunk))

                        # reset curr_input
                        curr_input = ""

                # leftover in curr_input
                if curr_input:
                    all_chunks.append(curr_input)

        # Now create rows for each chunk
        for chunk_i, chunk_text in enumerate(all_chunks):
            new_row = row.copy()
            new_row[text_col] = chunk_text
            new_row["chunk_id"] = chunk_i
            chunked_rows.append(new_row)

    chunked_df = pd.DataFrame(chunked_rows).reset_index(drop=True)
    return chunked_df


# ------------------------------------------------------------------------------
# 2) Filter rows that need processing (where "target" is NaN) and expand them
# ------------------------------------------------------------------------------
to_process_df = test_df[test_df["target"].isna()].reset_index()  # preserve original index in a column named "index"

expanded_df = chunk_large_texts(
    df=to_process_df,
    tokenizer=tokenizer,
    text_col="text",
    max_tokens=max_tokens
)

# ------------------------------------------------------------------------------
# 3) collate_fn, referencing the chunked text
# ------------------------------------------------------------------------------
def formatting_prompts_func(example):
    # You can adjust how you format your prompt here
    return example["text"]

def collate_fn(examples):
    indices = [ex["index"] for ex in examples]     # the original test_df row index
    chunk_ids = [ex["chunk_id"] for ex in examples]

    texts = [formatting_prompts_func(ex) for ex in examples]
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    tokenized["indices"] = indices
    tokenized["chunk_ids"] = chunk_ids
    return tokenized

# ------------------------------------------------------------------------------
# 4) Create DataLoader on expanded_df and run generation loop
# ------------------------------------------------------------------------------
dataloader = DataLoader(
    expanded_df.to_dict(orient="records"),
    batch_size=batch_size,
    collate_fn=collate_fn
)

processed_rows = 0

for batch in tqdm(dataloader):
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    prompt_padded_len = input_ids.shape[1]

    # Generate
    gen_tokens = model.generate(
        input_ids,
        attention_mask=attention_mask,
        use_cache=True,
        max_new_tokens=max_new_tokens
        # do_sample=True, # etc. if you want sampling
        # temperature=..., top_p=..., top_k=...
    )

    # Strip off the prompt tokens
    gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]

    # Decode
    corrections = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    # Optional tidy-up: corrections = ["".join(c) for c in corrections]

    # Put these chunk-level outputs into expanded_df
    for idx, cid, corr in zip(batch["indices"], batch["chunk_ids"], corrections):
        mask = (expanded_df["index"] == idx) & (expanded_df["chunk_id"] == cid)
        expanded_df.loc[mask, "target"] = corr
        processed_rows += 1

        # Partial save of the expanded_df for safety
        if processed_rows % save_each == 0:
            expanded_df.to_csv("expanded_temp.csv", index=False)
            print(f"Saved expanded_df progress after processing {processed_rows} chunk rows.")

# Final save of chunk-level expansions (optional)
expanded_df.to_csv("expanded_df_final.csv", index=False)
print("Saved expanded_df_final.csv.")


# ------------------------------------------------------------------------------
# 5) Recombine chunk outputs into test_df (one output per original row)
# ------------------------------------------------------------------------------
grouped = expanded_df.groupby("index")["target"].apply(list).reset_index(name="chunks")

for _, row in grouped.iterrows():
    orig_idx = row["index"]  # the real test_df index
    chunk_outputs = row["chunks"]
    # combine them with newlines or spaces
    combined_output = "\n".join(str(x) for x in chunk_outputs if pd.notnull(x))
    # store in test_df
    test_df.loc[orig_idx, "target"] = combined_output

# Final save of test_df
test_df.to_csv(out_path, index=False)
print("Final save complete!")


  0%|          | 0/18 [00:00<?, ?it/s]<ipython-input-96-fcb60d847550>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[idx, "target"] = corr
100%|██████████| 18/18 [55:53<00:00, 186.29s/it]

Final save complete!


In [ ]:
from google.colab import runtime
runtime.unassign()